In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [2]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [11]:
# MNIST nn layers, 3개의 층을 쌓음
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()  # activation 함수

In [12]:
# Initialization(초기화)
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[ 0.8753,  0.9227, -1.5062,  ...,  1.0818, -1.8967,  0.0592],
        [-1.3478,  0.8176, -0.0606,  ...,  0.8136, -1.8421,  1.4126],
        [ 0.8205, -0.2898, -2.2894,  ...,  0.3526,  0.0076, -0.0392],
        ...,
        [-1.9113,  2.0265, -0.3215,  ..., -0.5589,  0.7735,  1.1883],
        [-1.4636, -0.6124,  0.1951,  ...,  0.5883,  0.2072, -0.6541],
        [-0.1766,  1.2805, -1.0738,  ...,  1.5949, -1.7846, -0.8129]],
       requires_grad=True)

In [15]:
# model, 마지막 linear3에 relu를 쓰지 않은 것은 crossentropy를 쓸거라서!
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [16]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:

total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 157.358352661
Epoch: 0002 cost = 39.523742676
Epoch: 0003 cost = 24.797601700
Epoch: 0004 cost = 17.247320175
Epoch: 0005 cost = 12.502470970
Epoch: 0006 cost = 9.351479530
Epoch: 0007 cost = 6.834040642
Epoch: 0008 cost = 5.047572136
Epoch: 0009 cost = 3.805500746
Epoch: 0010 cost = 2.918148994
Epoch: 0011 cost = 2.066921711
Epoch: 0012 cost = 1.575472713
Epoch: 0013 cost = 1.178432703
Epoch: 0014 cost = 0.954208076
Epoch: 0015 cost = 0.718015790
Learning finished
